In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
def image_preprocess(img, skip_dilate=False):
    ''' Cnvert the image to gray scale, blur image, apply adaptive threshold to highlight main features of the image '''
        
    # Gaussian blur to image with kernal size of (9, 9)
    img_proc = cv2.GaussianBlur(img, (9, 9), 0)
    
    # Adaptive threshold using 11 nearest neighbour pixels
    img_proc = cv2.adaptiveThreshold(img_proc, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    
    # Invert colours, so gridlines have non-zero pixel values.
    # Necessary to dilate the image, otherwise will look like erosion instead.
    img_proc = cv2.bitwise_not(img_proc, img_proc) 
    
    if not skip_dilate:
        kernel = np.array([[0., 1., 0.], [1., 1., 1.], [0., 1., 0.]],np.uint8)
        img_proc = cv2.dilate(img_proc, kernel, iterations=1)
    
    return img_proc

In [3]:
def find_largest_contour_points(img):
    ''' find the largest contour in the image'''
    
    # Find contours in the image
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort the contours in descending order
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Draw contour in image
    #cv2.drawContours(contours[0], bigContour, -1, (255, 0, 0), 3)
    
    # Find the perimeter of the lagest contour
    perimeter = cv2.arcLength(contours[0], True)
    
    # Find the polygon details from the contour
    get_ploy = cv2.approxPolyDP(contours[0], 0.02 * perimeter, True)
    
    # Reorder Contour points
    points = reorder_points(get_ploy)
    
    return points

In [4]:
def reorder_points(points):
    ''' reorder contour points'''
    
    # reshape contour points array
    points = points.reshape((4,2))
    
    #print(f'Contour points : { points }')
    
    # array to hold re-ordered points
    points_new = np.zeros((4,1,2), np.int32)
    
    # (right, bottom) (left, top)
    add = points.sum(axis=1)
    points_new[0] = points[np.argmin(add)]
    points_new[2] = points[np.argmax(add)]
    
    # (lef, bottom) (right, top)
    diff = np.diff(points, axis = 1)
    points_new[1] = points[np.argmin(diff)]
    points_new[3] = points[np.argmax(diff)]
    
    return points_new

In [5]:
def calculate_distance(pt1, pt2):
    
    # calculate distance between two points
    distance = np.sqrt(((pt1[0][0] - pt2[0][0]) ** 2 ) + ((pt1[0][1] - pt2[0][1]) ** 2))
    #print(f'Distance calculated { distance }')
    
    return distance

In [6]:
def get_warp(image, contour_points):
    ''' function to corp and warp the image'''
    
    # calculate the maximum value of side length
    side = max([calculate_distance(contour_points[0], contour_points[1]),
               calculate_distance(contour_points[1], contour_points[2]),
               calculate_distance(contour_points[2], contour_points[3]),
               calculate_distance(contour_points[3], contour_points[0])])
    
    #print(f'Side Calculated : { side }')
    
    # points source array for perspective transformation
    pts1 = np.float32(contour_points)
    
    # points destination array for perspective transformation
    pts2 = np.float32([[0, 0], [int(side)-1, 0], [int(side)-1, int(side)-1], [0, int(side)-1]])
    
    # Gets the transformation matrix for skewing the image to fit a square by comparing the 4 before and after points
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    
    # Performs the transformation on the original image
    image_out = cv2.warpPerspective(image, matrix, (int(side), int(side)))
    
    return image_out

In [7]:
def get_digit_boxes(img):
    ''' function to find the corners of individual sqaures'''
    
    digit_boxes = []
    side_length = img.shape[:1]
    side_length = side_length[0] / 9
    
    # the rectangles are stored in the list reading left-right instead of top-down
    for j in range(9):
        for i in range(9):
            # Top left corner of a bounding box
            pt1 = (i * side_length, j * side_length)
            
            # Bottom right corner of bounding box
            pt2 = ((i +1 ) * side_length, (j + 1) * side_length)
            
            digit_boxes.append((pt1, pt2))
    
    return digit_boxes

In [8]:
def cut_from_rect(image, box):
    return image[int(box[0][1]):int(box[1][1]), int(box[0][0]):int(box[1][0])]

In [9]:
def find_largest_feature(digit, scan_top_left, scan_btm_rght):
    
    height, width = digit.shape[:2]
    
    max_area = 0
    seed_point = (None, None)
    
    if scan_top_left is None:
        scan_top_left = [0, 0]
        
    if scan_btm_rght is None:
        scan_btm_rght = [height, width]
        
    # Loop through the image
    for x in range(scan_top_left[0], scan_btm_rght[0]):
        for y in range(scan_top_left[1], scan_btm_rght[1]):
            if digit.item(y, x) == 255 and x < width and y < height:
                area = cv2.floodFill(digit, None, (x, y), 64)
                if area[0] > max_area:
                    max_area = area[0]
                    seed_point = (x, y)
                    
    # Colour everything grey
    for x in range(width):
        for y in range(height):
            if digit.item(y, x) == 255 and x < width and y < height:
                cv2.floodFill(digit, None, (x, y), 64)
    
    # Mask that is 2 pixels bigger than the image
    mask = np.zeros((height + 2, width + 2), np.uint8)
    
    # Highlight the main feature
    if all([p is not None for p in seed_point]):
        cv2.floodFill(digit, mask, seed_point, 255)
        
    top, bottom, left, right = height, 0, width, 0
    
    for x in range(width):
        for y in range(height):
            if digit.item(y, x) == 64:  # Hide anything that isn't the main feature
                cv2.floodFill(digit, mask, (x, y), 0)

            # Find the bounding parameters
            if digit.item(y, x) == 255:
                top = y if y < top else top
                bottom = y if y > bottom else bottom
                left = x if x < left else left
                right = x if x > right else right
    
    # bounding box
    bbox = [[left, top], [right, bottom]]
    
    return np.array(bbox, dtype='float32'), seed_point

In [10]:
def scale_and_centre(img, size, margin=0, background=0):
    """Scales and centres an image onto a new background square."""
    h, w = img.shape[:2]

    def centre_pad(length):
        """Handles centering for a given length that may be odd or even."""
        if length % 2 == 0:
            side1 = int((size - length) / 2)
            side2 = side1
        else:
            side1 = int((size - length) / 2)
            side2 = side1 + 1
        return side1, side2

    def scale(r, x):
        return int(r * x)

    if h > w:
        t_pad = int(margin / 2)
        b_pad = t_pad
        ratio = (size - margin) / h
        w, h = scale(ratio, w), scale(ratio, h)
        l_pad, r_pad = centre_pad(w)
    else:
        l_pad = int(margin / 2)
        r_pad = l_pad
        ratio = (size - margin) / w
        w, h = scale(ratio, w), scale(ratio, h)
        t_pad, b_pad = centre_pad(h)

    img = cv2.resize(img, (w, h))
    img = cv2.copyMakeBorder(img, t_pad, b_pad, l_pad, r_pad, cv2.BORDER_CONSTANT, None, background)
    return cv2.resize(img, (size, size))

In [11]:
def extract_digit(image, box, size):
    '''Extracts a digit (if one exists) from a Sudoku square.'''
    
    # Get the digit box from the whole square
    digit = cut_from_rect(image, box)
    
    #plt.imshow(digit)
    
    # use floodfill feature to find the largest feature in the rectange
    h, w = digit.shape[:2]
    margin = int(np.mean([h, w]) / 2.5)
    bbox, seed = find_largest_feature(digit, [margin, margin], [w - margin, h - margin])
    
    digit = cut_from_rect(digit, bbox)
    
    # Scale and pad the digit so that it fits a square of the digit size we're using for machine learning
    w = bbox[1][0] - bbox[0][0]
    h = bbox[1][1] - bbox[0][1]
    
    # Ignore any small bounding boxes
    if w > 0 and h > 0 and (w * h) > 100 and len(digit) > 0:
        return scale_and_centre(digit, size, 4)
    else:
        return np.zeros((size, size), np.uint8)
    

In [14]:
def get_digit_images(image, digit_boxes, size=28):
    
    digit_images = []
    
    image = image_preprocess(image, skip_dilate=True)
    
    for digit_box in digit_boxes:
        digit_images.append(extract_digit(image, digit_box, size))
        
    return digit_images

In [15]:
def show_digits(digits, colour=255):
    """Shows list of 81 extracted digits in a grid format"""
    rows = []
    with_border = [cv2.copyMakeBorder(img.copy(), 1, 1, 1, 1, cv2.BORDER_CONSTANT, None, colour) for img in digits]
    for i in range(9):
        row = np.concatenate(with_border[i * 9:((i + 1) * 9)], axis=1)
        rows.append(row)
    img = show_image(np.concatenate(rows))
    return img

In [31]:
def show_image(img):
    """Shows an image until any key is pressed"""
#    print(type(img))
#    print(img.shape)
    cv2.imshow('image', img)  # Display the image
    cv2.imwrite('gau_sudoku3.jpg', img)
    cv2.waitKey(0)  # Wait for any key to be pressed (with the image window active)
    cv2.destroyAllWindows()  # Close all windows
    return img

In [16]:
def process_digit_images(digit_images):
    k=0
    ratio = 6
    plt.figure(figsize=(10, 10))
    for digit_img in digit_images:
        #print(f'digit images : {digit_img.shape}')
        #h, w = digit_img.shape[:2]
        #print(f'height: {h} width: {w}')
        #image_crop = digit_img[ ratio : (int(h) - 6), ratio : (int(w) - 6)]
        plt.subplot(9, 9 , k+1)
        plt.imshow(digit_img)
        k= k+1
        
    plt.show()

In [17]:
def load_model():
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded saved model from disk.")
    
    return loaded_model

In [27]:
def predict_numbers(num_mat, model):
    numbers = []
    ratio = 6
    for img in num_mat:
        img = img.reshape(1, 28, 28, 1)
        number = np.argmax(model.predict(img , verbose = 0))
        print(f'Predicted number: {number}')
        numbers.append(number)
        
    return numbers

In [36]:
image = cv2.imread('Sudoku.jpeg', cv2.IMREAD_GRAYSCALE)

imgContour = image.copy()

# preprocess image
image_process = image_preprocess(image)

# find the corner points of largest image
contour_points = find_largest_contour_points(image_process)

# crop and warp image
image_cropped = get_warp(image, contour_points)

# find the corners of individual digit boxes in puzzle
digit_boxes = get_digit_boxes(image_cropped)

digit_images = get_digit_images(image_cropped, digit_boxes, size=28)

final_image = show_digits(digit_images)

#process_digit_images(digit_images)

#model = load_model()

#puzzle_numbers = predict_numbers(digit_images, model)

#print(f'Puzzle Nummbers: {puzzle_numbers}')

#cv2.imshow('Sudoku Puzzle', image_cropeed)
#cv2.imshow('Canny Puzzle', imgContour)
#cv2.imshow('Corpped Puzzle', imgWarp)
#cv2.imwrite('Puzzle.jpg', imgWarp)


cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import model_from_json

In [41]:
def load_model():
    ''' function to load saved model '''
    
    json_file = open('model/model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model/model.h5")
    print("Loaded saved model from disk.")
    
    return loaded_model

In [36]:
def predict_numbers(digit, model):
    ''' function to predict the number from the digit image '''
    
    number = np.argmax(model.predict(digit , verbose = 0))
    
    return number

In [42]:
def extract_number(image):
    image = cv2.resize(image, (450, 450))
    number_grid = np.zeros((9, 9))
    
    model = load_model()
    
    for i in range(9):
        for j in range(9):
            digit = image[i * 50 : (i +1) * 50, j * 50 : (j + 1) * 50]
            #cv2.imwrite(f'images/{i}_{j}.jpg', digit)
            
            if digit.sum() > 80000:
                #print(f'digit sum: {digit.sum()}')
                digit =  cv2.resize(digit, (28, 28))
                digit = digit.reshape(1, 28, 28, 1)
                prediction = predict_numbers(digit, model)
                number_grid[i][j] = prediction
            else:
                number_grid[i][j] = 0
    
    return number_grid

In [79]:
image = cv2.imread('images/extract_sudoku.jpg', cv2.IMREAD_GRAYSCALE)
grid = extract_number(image)

print(grid)

Loaded saved model from disk.
[[0. 3. 3. 5. 0. 7. 0. 0. 0.]
 [2. 0. 2. 7. 9. 0. 0. 0. 0.]
 [0. 7. 0. 0. 0. 3. 0. 0. 0.]
 [3. 7. 0. 0. 3. 2. 2. 0. 0.]
 [0. 2. 3. 0. 0. 0. 3. 7. 2.]
 [0. 0. 3. 9. 0. 0. 0. 5. 0.]
 [7. 3. 0. 0. 7. 0. 3. 0. 0.]
 [3. 0. 0. 2. 0. 0. 7. 5. 0.]
 [2. 0. 7. 3. 5. 2. 5. 7. 0.]]


In [65]:
# A Backtracking program in Python to solve Sudoku problem 

# A Utility Function to print the Grid 
def print_grid(arr):
    
    for i in range(9):
        print(arr[i])
        #for j in range(9):
            #print (f'{arr[i][j]},')
        #print ('\n') 

# Function to Find the entry in the Grid that is still not used 
def find_empty_location(arr, l): 
    for row in range(9): 
        for col in range(9): 
            if(arr[row][col]== 0): 
                l[0]= row 
                l[1]= col 
                return True
    return False

# Returns a boolean which indicates whether any assigned entry 
# in the specified row matches the given number. 
def used_in_row(arr, row, num): 
    for i in range(9): 
        if(arr[row][i] == num): 
            return True
    return False

# Returns a boolean which indicates whether any assigned entry 
# in the specified column matches the given number. 
def used_in_col(arr, col, num): 
    for i in range(9): 
        if(arr[i][col] == num): 
            return True
    return False

# Returns a boolean which indicates whether any assigned entry 
# within the specified 3x3 box matches the given number 
def used_in_box(arr, row, col, num): 
    for i in range(3): 
        for j in range(3): 
            if(arr[i + row][j + col] == num): 
                return True
    return False

# Checks whether it will be legal to assign num to the given row, col 
# Returns a boolean which indicates whether it will be legal to assign 
# num to the given row, col location. 
def check_location_is_safe(arr, row, col, num): 
    
    # Check if 'num' is not already placed in current row, 
    # current column and current 3x3 box 
    return not used_in_row(arr, row, num) and not used_in_col(arr, col, num) and not used_in_box(arr, row - row % 3, col - col % 3, num) 

# Takes a partially filled-in grid and attempts to assign values to 
# all unassigned locations in such a way to meet the requirements 
# for Sudoku solution (non-duplication across rows, columns, and boxes) 
def solve_sudoku(arr): 
    
    # 'l' is a list variable that keeps the record of row and col in find_empty_location Function
    l =[0, 0] 
    
    # If there is no unassigned location, we are done	 
    if(not find_empty_location(arr, l)): 
        return True
    
    # Assigning list values to row and col that we got from the above Function 
    row = l[0]
    col = l[1] 
    
    # consider digits 1 to 9 
    for num in range(1, 10): 

        # if looks promising 
        if(check_location_is_safe(arr, row, col, num)): 
            
            # make tentative assignment 
            arr[row][col]= num 

            # return, if success, ya ! 
            if(solve_sudoku(arr)): 
                return True

            # failure, unmake & try again 
            arr[row][col] = 0
            
    # this triggers backtracking
    return False

# Driver main function to test above functions 
if __name__=="__main__": 
    
    # creating a 2D array for the grid 
    grid =[[0 for x in range(9)]for y in range(9)] 
    
    # assigning values to the grid 
    grid =[[3, 0, 6, 5, 0, 8, 4, 0, 0], 
        [5, 2, 0, 0, 0, 0, 0, 0, 0], 
        [0, 8, 7, 0, 0, 0, 0, 3, 1], 
        [0, 0, 3, 0, 1, 0, 0, 8, 0], 
        [9, 0, 0, 8, 6, 3, 0, 0, 5], 
        [0, 5, 0, 0, 9, 0, 6, 0, 0], 
        [1, 3, 0, 0, 0, 0, 2, 5, 0], 
        [0, 0, 0, 0, 0, 0, 0, 7, 4], 
        [0, 0, 5, 2, 0, 6, 3, 0, 0]] 
    
    # if success print the grid 
    if(solve_sudoku(grid)): 
        print_grid(grid) 
    else: 
        print ("No solution exists")


[3, 1, 6, 5, 7, 8, 4, 9, 2]
[5, 2, 9, 1, 3, 4, 7, 6, 8]
[4, 8, 7, 6, 2, 9, 5, 3, 1]
[2, 6, 3, 4, 1, 5, 9, 8, 7]
[9, 7, 4, 8, 6, 3, 1, 2, 5]
[8, 5, 1, 7, 9, 2, 6, 4, 3]
[1, 3, 8, 9, 4, 7, 2, 5, 6]
[6, 9, 2, 3, 5, 1, 8, 7, 4]
[7, 4, 5, 2, 8, 6, 3, 1, 9]


In [69]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPool2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train / 255
X_test = X_test / 255

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [75]:
# Build model
model = Sequential()

model.add(Flatten(input_shape=(28, 28)))

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 111,146
Trainable params: 111,146
Non-trainable params: 0
_________________________________________________________________


In [76]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

Epoch 1/10
300/300 [==============================] - 1s 3ms/step - loss: 0.4126 - accuracy: 0.8839 - val_loss: 0.1821 - val_accuracy: 0.9455
Epoch 2/10
300/300 [==============================] - 1s 3ms/step - loss: 0.1432 - accuracy: 0.9580 - val_loss: 0.1279 - val_accuracy: 0.9599
Epoch 3/10
300/300 [==============================] - 1s 3ms/step - loss: 0.1028 - accuracy: 0.9691 - val_loss: 0.0982 - val_accuracy: 0.9692
Epoch 4/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0788 - accuracy: 0.9764 - val_loss: 0.0913 - val_accuracy: 0.9714
Epoch 5/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0619 - accuracy: 0.9813 - val_loss: 0.0834 - val_accuracy: 0.9732
Epoch 6/10
300/300 [==============================] - 1s 2ms/step - loss: 0.0517 - accuracy: 0.9839 - val_loss: 0.0786 - val_accuracy: 0.9751
Epoch 7/10
300/300 [==============================] - 1s 3ms/step - loss: 0.0428 - accuracy: 0.9867 - val_loss: 0.0822 - val_accuracy: 0.9741
Epoch 

In [77]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print(f'Test accuracy : {test_acc}')

313/313 - 0s - loss: 0.0877 - accuracy: 0.9748
Test accuracy : 0.9747999906539917


In [78]:
# Save the model
# serialize model to JSON

model_json = model.to_json()
with open('model/model.json', 'w') as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights('model/model.h5')

print('Saved model to disk')

Saved model to disk
